In [ ]:
import os
os.environ['PIP_CACHE_DIR'] = "/content/drive/MyDrive/pip_cache"
os.environ['OPENAI_API_KEY'] = "xxxxxx"
!pip install langchain
!pip install openai
!pip install python-dotenv
!pip install chromadb
!pip install tiktoken

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

#docloaders depend on doctype
from langchain.document_loaders import WebBaseLoader


# llm and indexer
llm = ChatOpenAI(temperature=0, model_name="gpt-4-0613")
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter(
chunk_size=2000, chunk_overlap=0, separators=[" ", ",", "\n"]
)


# document source #1 goes here
loader1 = WebBaseLoader("https://en.wikipedia.org/wiki/Ultimate_Fighting_Championship")
docs1 = loader1.load()
source1 = text_splitter.split_documents(docs1)
source1_db = Chroma.from_documents(source1, embeddings, collection_name="ufc")
docchain1 = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=source1_db.as_retriever()
)


# document source #2 goes here
loader2 = WebBaseLoader("https://en.wikipedia.org/wiki/2022")
docs2 = loader2.load()
source2 = text_splitter.split_documents(docs2)
source2_db = Chroma.from_documents(source2, embeddings, collection_name="2022")
docchain2 = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=source2_db.as_retriever()
)

# Define some tools with which we will endow our agent
tools = [
    Tool(
        name="UFC system",
        func=docchain1.run,
        description="useful for when you need to answer questions about the Ultimate Fighting Championship. Input should be a fully formed question.",
        return_direct=True
    ),
    Tool(
        name="The year 2022",
        func=docchain2.run,
        description="useful for when you need to answer questions about things that happened in the year 2022. Input should be a fully formed question.",
        return_direct=True
    ),
]

# the agent is a router
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    "Who is a politician who died in the year 2022?"
)

In [ ]:
agent.run(
    "Has there ever been a 5-round non-title, non-main event bout?"
)